<a href="https://colab.research.google.com/github/MinsooKwak/Kaggle_note/blob/main/Data_science/big_data_derby_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고: https://www.kaggle.com/code/mattop/big-data-derby-2022-eda

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px # graphing
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import missingno as msno    # describe data
import os

from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

plt.rcParams["figure.figsize"]= (12,8)

for dirname, _ , filenames in os.walk('#경로 작성(big_data_derby_2022 이전까지)'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

In [ ]:
colors= ["#FFFFFF", "#6CD4FF", "#F7DF00", "#E60000"]
# color 시각화해보기

### dataset 설명